In [ ]:
from pathlib import Path
import random
import numpy as np
import bs4 as BeautifulSoup

from io import BytesIO
from PIL import Image as pi
from ipywidgets import HBox, VBox, Layout, HTML
from ipywidgets import Image as Image_widget
import colorlover as cl

import drawSvg as draw
import cairosvg

try:
    from fastai.vision import *
    from fastai.metrics import error_rate
    fastai_imported = True
except Exception as ex:
    print('Switch to fastapi-cpu kernel to train or use model.')
    fastai_imported = False

In [ ]:
path = Path('/Users/ray/data/PRImA_Layout_Analysis_Dataset/')
image_path = path / 'Images'
mask_path = path / 'XML'

In [ ]:
filenames = image_path.ls()
data_names = [f.stem for f in filenames]

    <element name="TextRegion" type="pc:TextRegionType"/>
    <element name="ImageRegion" type="pc:ImageRegionType"/>
    <element name="LineDrawingRegion" type="pc:LineDrawingRegionType"/>
    <element name="GraphicRegion" type="pc:GraphicRegionType"/>
    <element name="TableRegion" type="pc:TableRegionType"/>
    <element name="ChartRegion" type="pc:ChartRegionType"/>
    <element name="MapRegion" type="pc:MapRegionType"/>
    <element name="SeparatorRegion" type="pc:SeparatorRegionType"/>
    <element name="MathsRegion" type="pc:MathsRegionType"/>
    <element name="ChemRegion" type="pc:ChemRegionType"/>
    <element name="MusicRegion" type="pc:MusicRegionType"/>
    <element name="AdvertRegion" type="pc:AdvertRegionType"/>
    <element name="NoiseRegion" type="pc:NoiseRegionType"/>
    <element name="UnknownRegion" type="pc:UnknownRegionType"/>
    <element name="CustomRegion" type="pc:CustomRegionType"/>

In [ ]:
regions = ['textregion', 'imageregion', 'linedrawingregion', 'graphicregion', 'tableregion', 'chartregion', 
           'mapregion', 'separatorregion', 'mathsregion', 'chemregion', 'musicregion', 'advertregion', 
           'noiseregion', 'unknownregion', 'customregion']

In [ ]:
colors =['#e6194B', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#42d4f4', '#f032e6', 
         '#bfef45', '#fabebe', '#469990', '#e6beff', '#9A6324', '#fffac8', '#800000', '#aaffc3', 
         '#808000', '#ffd8b1', '#000075', '#a9a9a9', '#ffffff', '#000000']

In [ ]:
color_map = dict(zip(regions, colors))

In [ ]:
def xml_to_svg(data_name=None, alpha=1.0):
    image_filename =  image_path / (data_name + '.tif')
    image_xml = mask_path / (data_name + '.xml')
    if not image_xml.exists():
        image_xml = mask_path / ('pc-' + data_name + '.xml')
    
    doc_image = pi.open(image_filename)
    with open(image_xml) as fp:
        soup = BeautifulSoup.BeautifulSoup(fp)
        
    width, height = doc_image.size
    d = draw.Drawing(width, height, origin = (0, -height))
    used_regions = set()
    for region in regions:
        for tag in soup.find_all(region):
            count = 0
            # Without shape_rendering="crispEdges", cairosvg will anti-alias paths and break the 1-1
            # mapping of colors to pixel values for groundtruth image masks. Some test images that
            # display the effect w/out the crispEdges are in bad_greyscale.
            # bad_greyscale = ['00001203', '00001164', '00001131', '00000941', '00001131', '00001153', 
            #                  '00001287', '00001275']
            p = draw.Path(stroke_width=0, stroke=color_map[region],
                      fill=color_map[region], fill_opacity=alpha, shape_rendering="crispEdges")
            for t in tag.find_all('point'):
                used_regions.add(region)
                x = int(t['x'])
                y = -int(t['y'])
                if count == 0:
                    p.M(x, y)
                else:
                    p.L(x,y)
                count += 1
            p.Z()
            d.append(p)
            
    print(used_regions)
    return d.asSvg()


In [ ]:
def convert_image(data_name):
    svg = xml_to_svg(data_name)
    out = BytesIO()
    cairosvg.svg2png(svg, write_to=out)
    png = pi.open(out)
    return png.convert(mode='L')

In [ ]:
def show_mask(data_name):
    image_filename =  data_name + '.tif'    
    b = BytesIO()
    im = pi.open(image_path / image_filename)
    im.save(b, format='png')
    
    svg = xml_to_svg(data_name)
    png = cairosvg.svg2png(svg)
    mask = Image_widget(value=png, layout=Layout(width='40%', height='600px', border='1px solid black'))
    document_image = Image_widget(value=b.getvalue(), layout=Layout(width='40%', height='600px', 
                                                                    border='1px solid black'))
    return HBox([mask, document_image])

In [ ]:
png_data = np.asarray(convert_image('00001287'))
set([d for row in png_data for d in row])

In [ ]:
# Without shape_rendering="crispEdges", cairosvg will anti-alias paths and break the 1-1
# mapping of colors to pixel values for groundtruth image masks. Some test images that
# display the effect w/out the crispEdges are in bad_greyscale.
bad_greyscale = ['00001203', '00001164', '00001131', '00000941', '00001131', '00001153', 
                 '00001287', '00001275']
bad_groundtruth = ['00000272', '00001255', '00000726', '00000130', '00000127', '00000128',
                   '00000997', '00000925', '00001298', '00000874', '00001057', '00001107',
                   '00000657', '00000709', '00000820', '00001285', '00000662', '00000636',
                   '00000273', '00000989', '00001298', '00000087', '00001287', '00001289',
                   '00000826', '00000090', '00000657', '00001297', '00000699', '00001292',
                   '00000989', '00001288']
data_name = random.choice(data_names)
print(data_name)
png_data = np.asarray(convert_image(data_name))
print(set([d for row in png_data for d in row]))
show_mask(data_name)

In [ ]:
data_name = '00001115'
png_data = np.asarray(convert_image(data_name))
print(set([d for row in png_data for d in row]))
show_mask(data_name)

In [ ]:
data_name = '00001203'
svg = xml_to_svg(data_name=data_name, alpha=0.4)
out = BytesIO()
cairosvg.svg2png(svg, write_to=out)
png = pi.open(out)

In [ ]:
image_filename =  data_name + '.tif'    
b = BytesIO()
im = pi.open(image_path / image_filename)
im.save(b, format='png')

In [ ]:
background = im.convert("RGBA")
foreground = png.convert("RGBA")
merged = pi.alpha_composite(background, foreground)
merged